# 차기 신도시 아파트 공급을 위한 경기도 외각지역 위치 추천 &연도별 아파트 면적 선호도 추이
## 데이터 분석 및 시각화 과정 세미 프로젝트(M조) : 

## 인프라 접근성 지표 가공

**가공 목적**

- 공공데이터에서 행정구역,시설,시간대,교통수단별 접근가능 시설수의 데이터는 새롭게 정의할 인프라 지수를 대표하지 못함


**처리 사항**

1. 필요없는 칼럼들을 제거
2. 시간 구간별 가중치 칼럼별로 차등 부여
3. 시설물 대분류별 그룹화후 시설별 가중치를 따로 부여
4. 같은 행정구역읍면동을 가진 관찰값들의 총합을 구하고 새로운 데이터 프레임 형성
5. 새로운 데이터 프레임을 내림차순으로 정렬 



**활용 데이터**
- KOSIS : 행정구역,시설,시간대,교통수단별 접근가능시설수
- KOSIS : '현거주지 선택이유 주된 응답 자료'를 기반으로 산출한 가중치들

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd

In [2]:
# 실행마다 결과 보기
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1. 데이터 불러오기

In [3]:
# 데이터 불러오기
data=pd.read_excel("Pdata/행정구역,시설,시간대,교통수단별 접근가능시설수(2020년기준).xlsx")
data

,Year,행정구역코드,동/읍면구분코드,행정구역수준,시설물코드,출발시간대코드,교통수단코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,동/읍면구분명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
0,2020,0,-,0,11,0_AllDay,1_PC,전국,-,-,-,교육시설,초등학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
1,2020,0,-,0,11,0_AllDay,2_PT,전국,-,-,-,교육시설,초등학교,일평균(06-20시),대중교통/도보,5.095999,>10.00,>10.00,>10.00
2,2020,0,-,0,11,1_T0709,1_PC,전국,-,-,-,교육시설,초등학교,오전첨두(07-09시),승용차,>10.00,>10.00,>10.00,>10.00
3,2020,0,-,0,11,1_T0709,2_PT,전국,-,-,-,교육시설,초등학교,오전첨두(07-09시),대중교통/도보,5.32067,>10.00,>10.00,>10.00
4,2020,0,-,0,11,2_T1214,1_PC,전국,-,-,-,교육시설,초등학교,낮시간(12-14시),승용차,>10.00,>10.00,>10.00,>10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347419,2020,3902062,-,4,43,1_T0709,2_PT,제주특별자치도,서귀포시,예래동,-,광역교통시설,공항,오전첨두(07-09시),대중교통/도보,0,0,0,0.246685
347420,2020,3902062,-,4,43,2_T1214,1_PC,제주특별자치도,서귀포시,예래동,-,광역교통시설,공항,낮시간(12-14시),승용차,0,0,0,0.454144
347421,2020,3902062,-,4,43,2_T1214,2_PT,제주특별자치도,서귀포시,예래동,-,광역교통시설,공항,낮시간(12-14시),대중교통/도보,0,0,0,0
347422,2020,3902062,-,4,43,3_T1820,1_PC,제주특별자치도,서귀포시,예래동,-,광역교통시설,공항,저녁첨두(18-20시),승용차,0,0,0,0.454144


### pd.read_excel과 pd.read_csv의 작은 차이점

- csv
    - 한글이 포함된 파일을 불러올 때 인코딩이 필요함


- excel
    - 한글이 포함된 파일을 인코딩 없이 불러와도 문제가 없음


## 2. 데이터 살펴보기

### value의 data type 과 shape 확인

In [4]:
# 데이터 정보 확인 데이터 타입 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347424 entries, 0 to 347423
Data columns (total 19 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Year      347424 non-null  int64 
 1   행정구역코드    347424 non-null  int64 
 2   동/읍면구분코드  347424 non-null  object
 3   행정구역수준    347424 non-null  int64 
 4   시설물코드     347424 non-null  int64 
 5   출발시간대코드   347424 non-null  object
 6   교통수단코드    347424 non-null  object
 7   행정구역시도명   347424 non-null  object
 8   행정구역시군구명  347424 non-null  object
 9   행정구역읍면동명  347424 non-null  object
 10  동/읍면구분명   347424 non-null  object
 11  시설물대분류    347424 non-null  object
 12  시설물소분류    347424 non-null  object
 13  출발시간대명    347424 non-null  object
 14  교통수단명     347424 non-null  object
 15  15분 이내    346768 non-null  object
 16  30분 이내    346768 non-null  object
 17  45분 이내    346768 non-null  object
 18  60분 이내    346768 non-null  object
dtypes: int64(4), object(15)
memory usage: 50.4+ MB


## 3. 데이터 프레임 변형

### 1) 경기도 행만 추출

In [5]:
# 관측값 경기도만 가지고 오기
data2= data[data["행정구역시도명"]=="경기도"]
data2

,Year,행정구역코드,동/읍면구분코드,행정구역수준,시설물코드,출발시간대코드,교통수단코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,동/읍면구분명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
1848,2020,31,-,1,11,0_AllDay,1_PC,경기도,-,-,-,교육시설,초등학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
1849,2020,31,-,1,11,0_AllDay,2_PT,경기도,-,-,-,교육시설,초등학교,일평균(06-20시),대중교통/도보,5.227607,>10.00,>10.00,>10.00
1850,2020,31,-,1,11,1_T0709,1_PC,경기도,-,-,-,교육시설,초등학교,오전첨두(07-09시),승용차,>10.00,>10.00,>10.00,>10.00
1851,2020,31,-,1,11,1_T0709,2_PT,경기도,-,-,-,교육시설,초등학교,오전첨두(07-09시),대중교통/도보,5.495732,>10.00,>10.00,>10.00
1852,2020,31,-,1,11,2_T1214,1_PC,경기도,-,-,-,교육시설,초등학교,낮시간(12-14시),승용차,>10.00,>10.00,>10.00,>10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191395,2020,3138041,-,4,43,1_T0709,2_PT,경기도,양평군,개군면,-,광역교통시설,공항,오전첨두(07-09시),대중교통/도보,0,0,0,0
191396,2020,3138041,-,4,43,2_T1214,1_PC,경기도,양평군,개군면,-,광역교통시설,공항,낮시간(12-14시),승용차,0,0,0,0.558649
191397,2020,3138041,-,4,43,2_T1214,2_PT,경기도,양평군,개군면,-,광역교통시설,공항,낮시간(12-14시),대중교통/도보,0,0,0,0
191398,2020,3138041,-,4,43,3_T1820,1_PC,경기도,양평군,개군면,-,광역교통시설,공항,저녁첨두(18-20시),승용차,0,0,0,0.558649


### 2) 불필요한 칼럼 제거

In [6]:
# 필요 컬럼 추출후 다시 컬럼 정의
data3 = data2[["행정구역코드","행정구역시도명","행정구역시군구명","행정구역읍면동명","시설물대분류","시설물소분류","출발시간대명","교통수단명","15분 이내","30분 이내","45분 이내", "60분 이내"]]
data3.head()

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
1848,31,경기도,-,-,교육시설,초등학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
1849,31,경기도,-,-,교육시설,초등학교,일평균(06-20시),대중교통/도보,5.227607,>10.00,>10.00,>10.00
1850,31,경기도,-,-,교육시설,초등학교,오전첨두(07-09시),승용차,>10.00,>10.00,>10.00,>10.00
1851,31,경기도,-,-,교육시설,초등학교,오전첨두(07-09시),대중교통/도보,5.495732,>10.00,>10.00,>10.00
1852,31,경기도,-,-,교육시설,초등학교,낮시간(12-14시),승용차,>10.00,>10.00,>10.00,>10.00


### 3) 불필요한 행 제거

In [7]:
# 경기도 전체 통계, 경기도 시 통계 제외
data4=data3[data3["행정구역읍면동명"]!= "-"]
data4

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,7.489899,>10.00,>10.00,>10.00
142914,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,오전첨두(07-09시),승용차,>10.00,>10.00,>10.00,>10.00
142915,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,오전첨두(07-09시),대중교통/도보,7.937374,>10.00,>10.00,>10.00
142916,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,낮시간(12-14시),승용차,>10.00,>10.00,>10.00,>10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
191395,3138041,경기도,양평군,개군면,광역교통시설,공항,오전첨두(07-09시),대중교통/도보,0,0,0,0
191396,3138041,경기도,양평군,개군면,광역교통시설,공항,낮시간(12-14시),승용차,0,0,0,0.558649
191397,3138041,경기도,양평군,개군면,광역교통시설,공항,낮시간(12-14시),대중교통/도보,0,0,0,0
191398,3138041,경기도,양평군,개군면,광역교통시설,공항,저녁첨두(18-20시),승용차,0,0,0,0.558649


In [8]:
# 작은 단위 시간대 제거후 일평균만 만듬
data5=data4[data4["출발시간대명"]== "일평균(06-20시)"]
data5

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,7.489899,>10.00,>10.00,>10.00
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,3.16,>10.00,>10.00,>10.00
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,>10.00,>10.00,>10.00,>10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0,0,0.526639,0.872177
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0,2.237009,7.007235,>10.00
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0,0.099978,0.626617,1.400789
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0,0,0,0.558649


### 4) 연산을 위한 조작들

In [9]:
# 10 이상 값 10으로 바꾸기 
data6 = data5.replace({"15분 이내" : ">10.00"}, "10.00")
data6 = data6.replace({"30분 이내" : ">10.00"}, "10.00")
data6 = data6.replace({"45분 이내" : ">10.00"}, "10.00")
data6 = data6.replace({"60분 이내" : ">10.00"}, "10.00")
data6

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,10.00,10.00,10.00,10.00
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,7.489899,10.00,10.00,10.00
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,10.00,10.00,10.00,10.00
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,3.16,10.00,10.00,10.00
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,10.00,10.00,10.00,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0,0,0.526639,0.872177
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0,2.237009,7.007235,10.00
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0,0.099978,0.626617,1.400789
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0,0,0,0.558649


In [10]:
#시간 값이 NaN 인 경우 인구가 없기 때문에 제거
data7=data6.dropna(axis=0)
data7

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,10.00,10.00,10.00,10.00
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,7.489899,10.00,10.00,10.00
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,10.00,10.00,10.00,10.00
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,3.16,10.00,10.00,10.00
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,10.00,10.00,10.00,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0,0,0.526639,0.872177
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0,2.237009,7.007235,10.00
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0,0.099978,0.626617,1.400789
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0,0,0,0.558649


In [11]:
# 데이터 타입변환
data8=data7.astype({"15분 이내":"float"})
data8=data8.astype({"30분 이내":"float"})
data8=data8.astype({"45분 이내":"float"})
data8=data8.astype({"60분 이내":"float"})
data8.info()
data8

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12078 entries, 142912 to 191393
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   행정구역코드    12078 non-null  int64  
 1   행정구역시도명   12078 non-null  object 
 2   행정구역시군구명  12078 non-null  object 
 3   행정구역읍면동명  12078 non-null  object 
 4   시설물대분류    12078 non-null  object 
 5   시설물소분류    12078 non-null  object 
 6   출발시간대명    12078 non-null  object 
 7   교통수단명     12078 non-null  object 
 8   15분 이내    12078 non-null  float64
 9   30분 이내    12078 non-null  float64
 10  45분 이내    12078 non-null  float64
 11  60분 이내    12078 non-null  float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.2+ MB


,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,10.000000,10.000000,10.000000,10.000000
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,7.489899,10.000000,10.000000,10.000000
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,10.000000,10.000000,10.000000,10.000000
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,3.160000,10.000000,10.000000,10.000000
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,10.000000,10.000000,10.000000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0.000000,0.000000,0.526639,0.872177
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0.000000,2.237009,7.007235,10.000000
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0.000000,0.099978,0.626617,1.400789
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0.000000,0.000000,0.000000,0.558649


## 3. 가중치 부여를 위한 그룹화 와 가중치 부여

### 1) 시간에 가중치 부여 후 합 칼럼 생성

In [12]:
# 시간 자체에 가중치 부여
#주의 여러번 런 하지 말것
data8["15분 이내"]=data8["15분 이내"]*4
data8["30분 이내"]=data8["30분 이내"]*3
data8["45분 이내"]=data8["45분 이내"]*2
data9=data8
data9

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,29.959596,30.000000,20.000000,10.000000
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,12.640000,30.000000,20.000000,10.000000
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0.000000,0.000000,1.053278,0.872177
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0.000000,6.711028,14.014471,10.000000
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0.000000,0.299934,1.253234,1.400789
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0.000000,0.000000,0.000000,0.558649


In [13]:
# 15분 이내 부터 60분 이내 다 더하기
data9["Total"]=data9["15분 이내"]+data9["30분 이내"]+data9["45분 이내"]+data9["60분 이내"]
data10=data9
data10

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000,100.000000
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,29.959596,30.000000,20.000000,10.000000,89.959596
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000,100.000000
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,12.640000,30.000000,20.000000,10.000000,72.640000
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0.000000,0.000000,1.053278,0.872177,1.925455
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0.000000,6.711028,14.014471,10.000000,30.725499
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0.000000,0.299934,1.253234,1.400789,2.953957
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0.000000,0.000000,0.000000,0.558649,0.558649


### 2) 유사 시설들 통합

In [14]:
# 의료시설과 판매시설을 근린시설로 통합
data11=data10.replace("의료시설", "근린시설")
data11=data11.replace("판매시설", "근린시설")
data11

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000,100.000000
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,29.959596,30.000000,20.000000,10.000000,89.959596
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000,100.000000
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,12.640000,30.000000,20.000000,10.000000,72.640000
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191377,3138041,경기도,양평군,개군면,광역교통시설,버스터미널,일평균(06-20시),대중교통/도보,0.000000,0.000000,1.053278,0.872177,1.925455
191384,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),승용차,0.000000,6.711028,14.014471,10.000000,30.725499
191385,3138041,경기도,양평군,개군면,광역교통시설,철도역,일평균(06-20시),대중교통/도보,0.000000,0.299934,1.253234,1.400789,2.953957
191392,3138041,경기도,양평군,개군면,광역교통시설,공항,일평균(06-20시),승용차,0.000000,0.000000,0.000000,0.558649,0.558649


### 3) 교통수단별 그룹화

In [15]:
#교통수단명 그룹화
data11.교통수단명.unique()
grouped = data11.groupby(['교통수단명'])
group_car = grouped.get_group('승용차')
group_bus = grouped.get_group('대중교통/도보')
group_car.head()

array(['승용차', '대중교통/도보'], dtype=object)

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
142936,3101154,경기도,수원시 장안구,파장동,근린시설,공공의료시설,일평균(06-20시),승용차,8.928391,30.0,20.0,10.0,68.928391
142944,3101154,경기도,수원시 장안구,파장동,근린시설,병·의원,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000


### 4) 교통수단별 그룹화된 집단을 다시 가중치를 부여할 시설별로 그룹화

In [16]:
# 승용차 _ 시설물대분류 그룹화
group_car.시설물대분류.unique()
grouped2 = group_car.groupby(['시설물대분류'])
group_car_edu = grouped2.get_group('교육시설')
# group_car_hos = grouped2.get_group('의료시설')
# group_car_store = grouped2.get_group('판매시설')
group_car_con = grouped2.get_group('근린시설')
group_car_tra = grouped2.get_group('광역교통시설')
group_car_edu.head(3)
group_car_con.head()

array(['교육시설', '근린시설', '광역교통시설'], dtype=object)

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142912,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.0,30.0,20.0,10.0,100.0
142920,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),승용차,40.0,30.0,20.0,10.0,100.0
142928,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),승용차,40.0,30.0,20.0,10.0,100.0


,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142936,3101154,경기도,수원시 장안구,파장동,근린시설,공공의료시설,일평균(06-20시),승용차,8.928391,30.0,20.0,10.0,68.928391
142944,3101154,경기도,수원시 장안구,파장동,근린시설,병·의원,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
142952,3101154,경기도,수원시 장안구,파장동,근린시설,종합병원,일평균(06-20시),승용차,6.059137,30.0,20.0,10.0,66.059137
142960,3101154,경기도,수원시 장안구,파장동,근린시설,대규모점포,일평균(06-20시),승용차,32.449019,30.0,20.0,10.0,92.449019
142968,3101154,경기도,수원시 장안구,파장동,근린시설,전통시장,일평균(06-20시),승용차,32.892979,30.0,20.0,10.0,92.892979


In [17]:
# 대중교통/도보 _ 시설물대분류 그룹화

grouped3 = group_bus.groupby(['시설물대분류'])
group_bus_edu = grouped3.get_group('교육시설')
#group_hos2 = grouped3.get_group('의료시설')
#group_store2 = grouped3.get_group('판매시설')
group_bus_con = grouped3.get_group('근린시설')
group_bus_tra = grouped3.get_group('광역교통시설')
group_bus_edu.head(3)
group_bus_con.head()

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142913,3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),대중교통/도보,29.959596,30.0,20.0,10.0,89.959596
142921,3101154,경기도,수원시 장안구,파장동,교육시설,중학교,일평균(06-20시),대중교통/도보,12.640000,30.0,20.0,10.0,72.640000
142929,3101154,경기도,수원시 장안구,파장동,교육시설,고등학교,일평균(06-20시),대중교통/도보,10.998236,30.0,20.0,10.0,70.998236


,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
142937,3101154,경기도,수원시 장안구,파장동,근린시설,공공의료시설,일평균(06-20시),대중교통/도보,2.774880,11.924248,20.0,10.0,44.699128
142945,3101154,경기도,수원시 장안구,파장동,근린시설,병·의원,일평균(06-20시),대중교통/도보,40.000000,30.000000,20.0,10.0,100.000000
142953,3101154,경기도,수원시 장안구,파장동,근린시설,종합병원,일평균(06-20시),대중교통/도보,0.601134,11.426821,20.0,10.0,42.027955
142961,3101154,경기도,수원시 장안구,파장동,근린시설,대규모점포,일평균(06-20시),대중교통/도보,7.986917,30.000000,20.0,10.0,67.986917
142969,3101154,경기도,수원시 장안구,파장동,근린시설,전통시장,일평균(06-20시),대중교통/도보,7.203838,30.000000,20.0,10.0,67.203838


### 5) 다른 차트에서 구한 시설별 가중치 적용

In [18]:
# 계산한 가중치
traffic_weighting = 0.5360824742268041
commercial_weighting = 0.1134020618556701
education_weighting = 0.35051546391752575

In [19]:
# 자가용에 대한 읍면동별 인프라 통합지수
a=education_weighting*group_car_edu.groupby("행정구역코드")["Total"].sum()
b=commercial_weighting*group_car_con.groupby("행정구역코드")["Total"].sum()
c=traffic_weighting*group_car_tra.groupby("행정구역코드")["Total"].sum()
infrastructure_car=pd.DataFrame({"infra_value_car":a+b+c})
infrastructure_car

,infra_value_car
행정구역코드,
3101154,191.574634
3101155,193.465578
3101156,192.412156
3101157,192.001353
3101158,190.573690
...,...
3138037,95.052703
3138038,116.705920
3138039,108.853510


In [20]:
# 대중교통에 대한 읍면동별 인프라 통합지수
a1=education_weighting*group_bus_edu.groupby("행정구역코드")["Total"].sum()
b1=commercial_weighting*group_bus_con.groupby("행정구역코드")["Total"].sum()
c1=traffic_weighting*group_bus_tra.groupby("행정구역코드")["Total"].sum()
infrastructure_bus=pd.DataFrame({"infra_value_bus":a1+b1+c1})
infrastructure_bus

,infra_value_bus
행정구역코드,
3101154,141.725282
3101155,138.227763
3101156,141.552647
3101157,152.655461
3101158,148.927562
...,...
3138037,13.118907
3138038,10.541615
3138039,16.944929


## 4. 결론 도출을 위한 데이터 프레임 생성

### 1) 3종류의 인프라 지수 정보를 담은 데이터 프레임 생성

In [21]:
# 승용차+대중교통 데이터 프레임 합치기
# 자가용과 대중교통 인프라 지수 합친 값 칼럼에 추가
infrastructure_both = pd.concat([infrastructure_car,infrastructure_bus],axis=1)
infrastructure_both['infra_value_both'] = infrastructure_both['infra_value_car']+infrastructure_both['infra_value_bus']
infrastructure_both

,infra_value_car,infra_value_bus,infra_value_both
행정구역코드,,,
3101154,191.574634,141.725282,333.299916
3101155,193.465578,138.227763,331.693341
3101156,192.412156,141.552647,333.964804
3101157,192.001353,152.655461,344.656815
3101158,190.573690,148.927562,339.501252
...,...,...,...
3138037,95.052703,13.118907,108.171610
3138038,116.705920,10.541615,127.247536
3138039,108.853510,16.944929,125.798439


### 2) 상위 데이터 프레임과 concat할 데이터 프레임 생성과정

In [22]:
# 자가용 교육시설로만 묶인 그룹에서 행정구역 코드가 같은 그룹 제거
new_group_car = group_car_edu.drop_duplicates(subset = ['행정구역코드'])
new_group_car.set_index('행정구역코드', inplace=True)
new_group_car

,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total
행정구역코드,,,,,,,,,,,,
3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
3101155,경기도,수원시 장안구,율천동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
3101156,경기도,수원시 장안구,정자1동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
3101157,경기도,수원시 장안구,정자2동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
3101158,경기도,수원시 장안구,영화동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3138037,경기도,양평군,청운면,교육시설,초등학교,일평균(06-20시),승용차,9.631579,30.0,20.0,10.0,69.631579
3138038,경기도,양평군,양동면,교육시설,초등학교,일평균(06-20시),승용차,4.304762,30.0,20.0,10.0,64.304762
3138039,경기도,양평군,지평면,교육시설,초등학교,일평균(06-20시),승용차,9.016216,30.0,20.0,10.0,69.016216


### 3) 데이터 프레임 concat

In [23]:
# 중복되는 지역을 뺀 데이터 프레임과 우리가 구하고자 했던 인프라 지수 값 자가용, 대중교통, 합계 컬럼 axis=1방향으로 추가 
car_infra_result = pd.concat([new_group_car,infrastructure_car], axis=1)
car_infra_result
bus_infra_result = pd.concat([new_group_car,infrastructure_bus], axis=1)
bus_infra_result
both_infra_result = pd.concat([new_group_car,infrastructure_both], axis =1)
both_infra_result

,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total,infra_value_car
행정구역코드,,,,,,,,,,,,,
3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,191.574634
3101155,경기도,수원시 장안구,율천동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,193.465578
3101156,경기도,수원시 장안구,정자1동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,192.412156
3101157,경기도,수원시 장안구,정자2동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,192.001353
3101158,경기도,수원시 장안구,영화동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,190.573690
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138037,경기도,양평군,청운면,교육시설,초등학교,일평균(06-20시),승용차,9.631579,30.0,20.0,10.0,69.631579,95.052703
3138038,경기도,양평군,양동면,교육시설,초등학교,일평균(06-20시),승용차,4.304762,30.0,20.0,10.0,64.304762,116.705920
3138039,경기도,양평군,지평면,교육시설,초등학교,일평균(06-20시),승용차,9.016216,30.0,20.0,10.0,69.016216,108.853510


,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total,infra_value_bus
행정구역코드,,,,,,,,,,,,,
3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,141.725282
3101155,경기도,수원시 장안구,율천동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,138.227763
3101156,경기도,수원시 장안구,정자1동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,141.552647
3101157,경기도,수원시 장안구,정자2동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,152.655461
3101158,경기도,수원시 장안구,영화동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,148.927562
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138037,경기도,양평군,청운면,교육시설,초등학교,일평균(06-20시),승용차,9.631579,30.0,20.0,10.0,69.631579,13.118907
3138038,경기도,양평군,양동면,교육시설,초등학교,일평균(06-20시),승용차,4.304762,30.0,20.0,10.0,64.304762,10.541615
3138039,경기도,양평군,지평면,교육시설,초등학교,일평균(06-20시),승용차,9.016216,30.0,20.0,10.0,69.016216,16.944929


,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total,infra_value_car,infra_value_bus,infra_value_both
행정구역코드,,,,,,,,,,,,,,,
3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,191.574634,141.725282,333.299916
3101155,경기도,수원시 장안구,율천동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,193.465578,138.227763,331.693341
3101156,경기도,수원시 장안구,정자1동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,192.412156,141.552647,333.964804
3101157,경기도,수원시 장안구,정자2동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,192.001353,152.655461,344.656815
3101158,경기도,수원시 장안구,영화동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,190.573690,148.927562,339.501252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138037,경기도,양평군,청운면,교육시설,초등학교,일평균(06-20시),승용차,9.631579,30.0,20.0,10.0,69.631579,95.052703,13.118907,108.171610
3138038,경기도,양평군,양동면,교육시설,초등학교,일평균(06-20시),승용차,4.304762,30.0,20.0,10.0,64.304762,116.705920,10.541615,127.247536
3138039,경기도,양평군,지평면,교육시설,초등학교,일평균(06-20시),승용차,9.016216,30.0,20.0,10.0,69.016216,108.853510,16.944929,125.798439


### 4) 고려 범위 축소

- 비고려 지역 제거

In [24]:
# 기존 신도시와 개발중인 신도시 제거후 인프라 지수 합계를 기준으로 내림차순으로 정렬
code_drop_both_result = both_infra_result.drop(index=[3110451,3110452,3110452,3110457,3110355,3110361,3110455,3110456,3110353,3110356,3110357,3102362,3102363,3102364,3102360,3102361,3102358,3102359,3102351,3102352,3102353,3102354,3102355,3102356,3102377,3102378,3102371,3102367,3102372,3116054,3116055,3116056,3105089,3104259,
                                                      3126056,3126052,3126055,3126053,3126054,3126051,3120055,3120056,3120057,3120058,3120056,3102167,3118066,3102375,3102376,3102374,3102368,3124060,3124062,3124061,3107052,3107053,3107051,3107064,3107033,3124012,3124014,3124070,3124064,3124065,3124069,
                                                      3102375, 3102376, 3102374, 3102368, 3124061, 3124060, 3124062, 3124052, 3124064, 3124065, 3124070, 3124069, 3124071,3123056, 3123060,3123063,3123064,3101460,3101354,3101355,3119357,3119358,3119263,3119264,3126056,3126052,3126055,3126056,3126052,3126053,3126055,3126054,3126051,3102167,3118066,3107051,3107033,
                                                      3120056,3120055, 3120057,3120058,3120055,3120055,3120056,3120057,3120058
                                                     ])
code_drop_both_result.info()
code_drop_both_result.infra_value_both.sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 3101154 to 3138041
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   행정구역시도명           473 non-null    object 
 1   행정구역시군구명          473 non-null    object 
 2   행정구역읍면동명          473 non-null    object 
 3   시설물대분류            473 non-null    object 
 4   시설물소분류            473 non-null    object 
 5   출발시간대명            473 non-null    object 
 6   교통수단명             473 non-null    object 
 7   15분 이내            473 non-null    float64
 8   30분 이내            473 non-null    float64
 9   45분 이내            473 non-null    float64
 10  60분 이내            473 non-null    float64
 11  Total             473 non-null    float64
 12  infra_value_car   473 non-null    float64
 13  infra_value_bus   473 non-null    float64
 14  infra_value_both  473 non-null    float64
dtypes: float64(8), object(7)
memory usage: 59.1+ KB


행정구역코드
3104266    359.832645
3101253    356.541161
3101368    355.050567
3104265    353.394640
3104261    350.238431
              ...    
3135035     65.867592
3138032     62.134866
3135037     49.319588
3137035     39.626165
3120021     22.020619
Name: infra_value_both, Length: 473, dtype: float64

In [25]:
# 상위 3개만 보기
code_drop_both_result.head(3)

,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total,infra_value_car,infra_value_bus,infra_value_both
행정구역코드,,,,,,,,,,,,,,,
3101154,경기도,수원시 장안구,파장동,교육시설,초등학교,일평균(06-20시),승용차,40.0,30.0,20.0,10.0,100.0,191.574634,141.725282,333.299916
3101155,경기도,수원시 장안구,율천동,교육시설,초등학교,일평균(06-20시),승용차,40.0,30.0,20.0,10.0,100.0,193.465578,138.227763,331.693341
3101156,경기도,수원시 장안구,정자1동,교육시설,초등학교,일평균(06-20시),승용차,40.0,30.0,20.0,10.0,100.0,192.412156,141.552647,333.964804


In [26]:
# 수도권 제2 순환고속도로 밖 시, 군만 포함하기 위해 데이터 프레임 내의 시군구 값 확인 
code_drop_both_result["행정구역시군구명"].unique()

array(['수원시 장안구', '수원시 권선구', '수원시 팔달구', '수원시 영통구', '성남시 수정구', '성남시 중원구',
       '의정부시', '안양시 만안구', '안양시 동안구', '부천시', '광명시', '평택시', '동두천시',
       '안산시 상록구', '안산시 단원구', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시',
       '구리시', '남양주시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시 처인구',
       '용인시 기흥구', '용인시 수지구', '파주시', '이천시', '안성시', '김포시', '화성시', '광주시',
       '양주시', '포천시', '여주시', '연천군', '가평군', '양평군'], dtype=object)

- 고려 지역만 추출

In [27]:
# 경기도 외곽의 시, 군만 포함 데이터 프레임 만들기
area_total= code_drop_both_result[code_drop_both_result["행정구역시군구명"]=="연천군"]

for i in ["가평군","여주군","이천시","안성시","평택시"]:
    area = code_drop_both_result[code_drop_both_result["행정구역시군구명"]==i]
    area_total=pd.concat([area_total,area])
area_total

,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total,infra_value_car,infra_value_bus,infra_value_both
행정구역코드,,,,,,,,,,,,,,,
3135011,경기도,연천군,연천읍,교육시설,초등학교,일평균(06-20시),승용차,16.989247,30.0,20.0,10.0,76.989247,89.953657,40.579275,130.532932
3135012,경기도,연천군,전곡읍,교육시설,초등학교,일평균(06-20시),승용차,23.365022,30.0,20.0,10.0,83.365022,106.864432,53.226582,160.091014
3135031,경기도,연천군,군남면,교육시설,초등학교,일평균(06-20시),승용차,19.032258,30.0,20.0,10.0,79.032258,88.585031,20.447981,109.033011
3135032,경기도,연천군,청산면,교육시설,초등학교,일평균(06-20시),승용차,18.246575,30.0,20.0,10.0,78.246575,104.123418,38.018980,142.142398
3135033,경기도,연천군,백학면,교육시설,초등학교,일평균(06-20시),승용차,7.517241,30.0,20.0,10.0,67.517241,72.275975,17.622538,89.898513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107060,경기도,평택시,원평동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,184.514287,146.291810,330.806097
3107061,경기도,평택시,통복동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,180.355629,153.331884,333.687513
3107062,경기도,평택시,비전1동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.0,20.0,10.0,100.000000,182.193827,119.418341,301.612168


### 5) 상위 순서대로 정렬

In [28]:
# 내림차순으로 정렬
area_total = area_total.sort_values(by='infra_value_both',ascending=False)

In [29]:
# 행정구역 코드 칼럼으로 보내기
last=area_total.reset_index()
last

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,시설물대분류,시설물소분류,출발시간대명,교통수단명,15분 이내,30분 이내,45분 이내,60분 이내,Total,infra_value_car,infra_value_bus,infra_value_both
0,3107059,경기도,평택시,신평동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.0,100.000000,183.461809,151.290526,334.752335
1,3107061,경기도,평택시,통복동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.0,100.000000,180.355629,153.331884,333.687513
2,3107060,경기도,평택시,원평동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.0,100.000000,184.514287,146.291810,330.806097
3,3107065,경기도,평택시,비전2동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.0,100.000000,182.458699,136.481260,318.939959
4,3107055,경기도,평택시,지산동,교육시설,초등학교,일평균(06-20시),승용차,40.000000,30.000000,20.000000,10.0,100.000000,177.316690,135.666593,312.983283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,3135038,경기도,연천군,장남면,교육시설,초등학교,일평균(06-20시),승용차,4.000000,30.000000,20.000000,10.0,64.000000,69.670103,0.680412,70.350515
59,3135034,경기도,연천군,미산면,교육시설,초등학교,일평균(06-20시),승용차,5.875000,27.750000,20.000000,10.0,63.625000,66.705977,2.489552,69.195529
60,3135035,경기도,연천군,왕징면,교육시설,초등학교,일평균(06-20시),승용차,7.076923,22.615385,20.000000,10.0,59.692308,57.912424,7.955168,65.867592
61,3135037,경기도,연천군,중면,교육시설,초등학교,일평균(06-20시),승용차,4.000000,18.000000,20.000000,10.0,52.000000,49.206186,0.113402,49.319588


### 6) 칼럼 정리

In [30]:
# 불필요한 칼럼 드롭
last=last.drop(["출발시간대명","교통수단명","15분 이내","30분 이내", "45분 이내","60분 이내", "Total","시설물대분류","시설물소분류"],axis=1)
last

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,infra_value_car,infra_value_bus,infra_value_both
0,3107059,경기도,평택시,신평동,183.461809,151.290526,334.752335
1,3107061,경기도,평택시,통복동,180.355629,153.331884,333.687513
2,3107060,경기도,평택시,원평동,184.514287,146.291810,330.806097
3,3107065,경기도,평택시,비전2동,182.458699,136.481260,318.939959
4,3107055,경기도,평택시,지산동,177.316690,135.666593,312.983283
...,...,...,...,...,...,...,...
58,3135038,경기도,연천군,장남면,69.670103,0.680412,70.350515
59,3135034,경기도,연천군,미산면,66.705977,2.489552,69.195529
60,3135035,경기도,연천군,왕징면,57.912424,7.955168,65.867592
61,3135037,경기도,연천군,중면,49.206186,0.113402,49.319588


In [31]:
# 인덱스 이름부여
last.index.name="순위"
last

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,infra_value_car,infra_value_bus,infra_value_both
순위,,,,,,,
0,3107059,경기도,평택시,신평동,183.461809,151.290526,334.752335
1,3107061,경기도,평택시,통복동,180.355629,153.331884,333.687513
2,3107060,경기도,평택시,원평동,184.514287,146.291810,330.806097
3,3107065,경기도,평택시,비전2동,182.458699,136.481260,318.939959
4,3107055,경기도,평택시,지산동,177.316690,135.666593,312.983283
...,...,...,...,...,...,...,...
58,3135038,경기도,연천군,장남면,69.670103,0.680412,70.350515
59,3135034,경기도,연천군,미산면,66.705977,2.489552,69.195529
60,3135035,경기도,연천군,왕징면,57.912424,7.955168,65.867592


### 7) 결과 보기

In [32]:
# 상위 20개의 읍,면,동 확인
last.iloc[0:20]

,행정구역코드,행정구역시도명,행정구역시군구명,행정구역읍면동명,infra_value_car,infra_value_bus,infra_value_both
순위,,,,,,,
0,3107059,경기도,평택시,신평동,183.461809,151.290526,334.752335
1,3107061,경기도,평택시,통복동,180.355629,153.331884,333.687513
2,3107060,경기도,평택시,원평동,184.514287,146.291810,330.806097
3,3107065,경기도,평택시,비전2동,182.458699,136.481260,318.939959
4,3107055,경기도,평택시,지산동,177.316690,135.666593,312.983283
5,3107062,경기도,평택시,비전1동,182.193827,119.418341,301.612168
6,3107056,경기도,평택시,송북동,178.847765,116.601717,295.449482
7,3107066,경기도,평택시,용이동,172.276732,119.437561,291.714293
8,3107058,경기도,평택시,신장2동,166.177455,121.317750,287.495206
